In [1]:
import pandas as pd
import numpy as np
import urllib
import io
import re
import os
import datetime

/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data_dir = 'var'
try:
    os.mkdir(data_dir)
except:
    pass

In [3]:
base='https://uk.wikisource.org'

In [4]:
archive='Архів:ДАДнО/Д'

In [5]:
def make_archive_url(base, archive):
    return base + '/wiki/' + str(urllib.parse.quote(archive))

In [6]:
def pick_table(tables, column_name, column_index=0):
    for table in tables:
        try:
            if table.columns[0][column_index] == column_name:
                return table
        except:
            pass
    return None

In [7]:
def load_archive(archive, base=base):
    df = pd.read_html(make_archive_url(base, archive), extract_links="all")
    return pick_table(df, '№')

In [8]:
archive_db = load_archive(archive)

In [9]:
def archive_as_csv(dataframe, stream=None):
    do_print = False
    if stream is None:
        stream = io.StringIO()
        do_print = True
    cols = dataframe.columns
    cols = [col[0] for col in cols]
    cols.insert(1, 'Link')
    stream.write(','.join(cols) + '\n')
    table = dataframe.to_numpy()
    table = table.tolist()
    for row in table:
        #print(row)
        fixed_row = list(row[0]) + [x[0] for x in row[1:]]
        fixed_row = map(lambda x: '' if 'redlink' in x else x, fixed_row)
        stream.write(','.join(fixed_row) + '\n')
    if do_print:
        print(stream.getvalue())

In [10]:
archive_as_csv(archive_db)

№,Link,Назва фонду,Крайні дати,Справ
11,,Канцелярія Катеринославського губернатора, м. Катеринослав,,
20,,Катеринославське губернське правління, м. Катеринослав,,
23,,Старший фабричний інспектор Катеринославської губернії, м. Катеринослав,,
33,,Катеринославське губернське у справах товариств та союзів присутствіє, м. Катеринослав,,
34,/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/34,,,
41,,Новомосковський повітовий землемір Катеринославської губернії,,
54,,Катеринославське губернське з військової повинності присутствіє, м. Катеринослав,,
70,,Катеринославська казенна палата, м. Катеринослав,,
72,,Катеринославська палата цивільного суду, м. Катеринослав,,
104,/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/104,Катеринославське духовне правління, м. Катеринослав,1768-1784,74
106,,Катеринославська духовна консисторія, м. Катеринослав,,
113,,Катеринославська губернська тюремна інспекція, м. Катеринослав,,
120,/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0

In [11]:
def fonds_from_archive(dataframe, only_linked=True):
    table = dataframe.to_numpy()
    table = table.tolist()
    result = [row[0] for row in table]
    result = map(lambda x: (x[0], None) if 'redlink' in x[1] else x, result)
    if only_linked:
        result = filter(lambda x: x[1] is not None, result)
    return list(result)

In [12]:
fonds = fonds_from_archive(archive_db)

In [13]:
def load_fond(url, base=base):
    df = pd.read_html(base + '/' + url, extract_links="all")
    return pick_table(df, 'Опис')

In [14]:
fond = load_fond(fonds[0][1])

In [15]:
archive_as_csv(fond)

Опис,Link,Назва,Роки,Справ
726,/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/34/726,Опис 726,,



In [16]:
def opi_from_fond(dataframe, only_linked=True):
    return fonds_from_archive(dataframe)

In [17]:
def cases_from_opus(dataframe, only_linked=True):
    return fonds_from_archive(dataframe)

In [18]:
opi=opi_from_fond(fond)

In [19]:
opi

[('726',
  '/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/34/726')]

In [20]:
def load_opus(url, base=base):
    df = pd.read_html(base + '/' + url, extract_links="all")
    return pick_table(df, '№')

In [21]:
opus = load_opus(opi[0][1])

In [22]:
archive_as_csv(opus)

№,Link,Назва,Роки,Сторінки
5498,/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/34/726/5498,Список избирателей евреев по Екатеринославу в 4 Государственную Думу,1912,



In [23]:
cases = cases_from_opus(opus)

In [24]:
cases

[('5498',
  '/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%94%D0%BD%D0%9E/34/726/5498')]

In [25]:
def lastmod(url):
    file = urllib.request.urlopen(url)
    for line in file:
        #print(line)
        if b'lastmod' in line:
            line = line.decode('utf-8')
            pattern = re.compile('[0-9][0-9]:[0-9][0-9].+[0-9][0-9]?.+[0-9][0-9][0-9][0-9]')
            result = re.search(pattern, line)
            if result is not None:
                return result.group(0)
            else:
                return line
    return None

In [26]:
x = lastmod(base + opi[0][1])

In [27]:
print(x)

08:38, 24 травня 2022


In [28]:
def scan_opus(opus, fond_id, base, stream):
    stream.write(f'opus,{fond_id},')
    stream.write(','.join(opus))
    stream.write(',' + lastmod(base + opus[1]))
    stream.write('\n')
    opus_db = load_opus(opus[1], base=base)
    opus_id = opus[0]
    cases = cases_from_opus(opus_db)
    for case in cases:
        stream.write(f'case,{fond_id},{opus_id},')
        stream.write(','.join(case))
        stream.write(',' + lastmod(base + case[1]))
        stream.write('\n')

In [29]:
def scan_fond(fond, base, stream):
    stream.write('fond,')
    stream.write(','.join(fond))
    stream.write(',' + lastmod(base + fond[1]))
    stream.write('\n')
    fond_db = load_fond(fond[1], base=base)
    opi = opi_from_fond(fond_db)
    for opus in opi:
        scan_opus(opus, fond[0], base=base, stream=stream)

In [30]:
def scan_archive(archive, base=base, stream=None):
    do_print = False
    if stream is None:
        stream = io.StringIO()
        do_print = True
        
    print("scanning ", archive)
    url = make_archive_url(base, archive)
    #print(url)
    stream.write(','.join(['archive', archive, url, lastmod(url)]))
    stream.write('\n')
    
    archive_db = load_archive(archive, base=base)
    fonds = fonds_from_archive(archive_db)
    for fond in fonds:
        scan_fond(fond, base=base, stream=stream)
    
    if do_print:
        print(stream.getvalue())

In [31]:
scan_archive(archive)

scanning  Архів:ДАДнО/Д


KeyboardInterrupt: 

In [ ]:
str(datetime.datetime.now())

In [ ]:
fname = data_dir + '/' + str(datetime.datetime.now()) + '.csv'
with open(fname, 'w') as stream:
    scan_archive(archive, base=base, stream=stream)

In [ ]:
archive

In [32]:
import datetime

In [33]:
str(datetime.datetime.now())

'2023-03-28 13:34:39.677114'